In [37]:
import torch
import numpy as np
import laspy
import pandas as pd
import os

las_subsampled_path = '/data/sdd/training_v2_subsampled_centroid_0.05.las'
pth_input_path = '/home/sogilvy/Pointcept/notebooks/output_file_norm.pth'

In [30]:
def print_las_version(file_path):
    with laspy.open(file_path) as file:
        header = file.header
        # Access the version directly from the header
        version = header.version
        print(f"LAS file version: {version}")

# Replace 'path_to_your_file.las' with the path to your LAS file
print_las_version('/data/sdd/training_v2_subsampled_centroid_0.05.las')
print_las_version('/data/sdd/training_v2.las')

LAS file version: 1.4
LAS file version: 1.4


In [2]:
import torch
import numpy as np
import os

def partition_pth_file(input_pth_path, base_output_directory, num_voxels_x, num_voxels_y, num_voxels_z):
    # Load the data from the .pth file
    data = torch.load(input_pth_path)

    # Extract coordinates and convert to numpy for processing
    coords = data['coord']

    # Calculate the bounds and voxel sizes
    min_coords = coords.min(axis=0)
    max_coords = coords.max(axis=0)
    voxel_size = (max_coords - min_coords) / np.array([num_voxels_x, num_voxels_y, num_voxels_z])

    # Calculate voxel indices for each point
    voxel_indices = np.floor((coords - min_coords) / voxel_size).astype(int)
    voxel_indices = np.clip(voxel_indices, 0, [num_voxels_x-1, num_voxels_y-1, num_voxels_z-1])

    # Ensure the output directory exists
    os.makedirs(base_output_directory, exist_ok=True)

    # Loop over all voxels
    for ix in range(num_voxels_x):
        for iy in range(num_voxels_y):
            for iz in range(num_voxels_z):
                # Find points in this voxel
                voxel_mask = (voxel_indices[:, 0] == ix) & (voxel_indices[:, 1] == iy) & (voxel_indices[:, 2] == iz)
                if voxel_mask.any():
                    voxel_data = {key: data[key][voxel_mask] for key in data if isinstance(data[key], np.ndarray)}
                    voxel_data['scene_id'] = data['scene_id']  # Carry over the scene_id unchanged

                    # Path for the new split file
                    voxel_file_path = os.path.join(base_output_directory, f'voxel_{ix}_{iy}_{iz}.pth')
                    torch.save(voxel_data, voxel_file_path)

    print(f"All voxel files saved in {base_output_directory}")


In [4]:
# Example usage
num_vox = 6
num_voxels_x, num_voxels_y, num_voxels_z = num_vox, num_vox, num_vox  # Specify the number of voxels in each dimension
base_output_directory = f'/data/sdd/qh/vox{num_vox}res0.2'
pth_input_path = '/data/sdd/training_v2_subsampled_centroid_0.02.pth'
partition_pth_file(pth_input_path, base_output_directory, num_voxels_x, num_voxels_y, num_voxels_z)


All voxel files saved in /data/sdd/qh/vox6res0.2
